# Overview

This a notebook that inspects the results of a WarpX simulation.

# Instruction

Enter the path of the data you wish to visualize below. Then execute the cells one by one, by selecting them with your mouse and typing `Shift + Enter`

In [ ]:
data_path = '../Example/Langmuir/'

In [ ]:
# Import statements
import os, glob
import yt ; yt.funcs.mylog.setLevel(50)
from IPython.display import clear_output
import numpy as np
from ipywidgets import interact, ToggleButtons, IntSlider
import matplotlib.pyplot as plt
%matplotlib

In [ ]:
# YT basic usage. Read this for practical examples
# ------------------------------------------------

# Create a dataset object
ds = yt.load( './pml/plt00400' )
# List all fields in the datasert
ds.field_list
# Get All Data from the dataset
ad = ds.all_data()
# Get some data. ".v" converts arrays from units-aware yt arrays to numpy arrays.
# Magnetic field
b = ad['boxlib', 'Bx'].v
# particle weight and z coordinate for species particle0
w = ad['particle0', 'particle_momentum_z'].v
z = ad['particle0', 'particle_weight'].v

# Create a sliceplot object
sl = yt.SlicePlot(ds, 2, 'Ex')
# Set labels
sl.set_xlabel(r'$x (\mu m)$')
sl.set_ylabel(r'$z (\mu m)$')
# Set figure size
sl.figure_size = (9, 7)
# Plot particles
sl.annotate_particles(width=1., p_size=2, ptype='particle0', col='black')
# Show grids
sl.annotate_grids()
# show the plot
sl.show()
# Save image
# sl.save('./toto.pdf')
# This returns the domain boundaries
sl.bounds

# Functions to plot the fields

In [5]:
def plot_field( iteration, field, slicing_direction='y', plotter='matplotlib' ):
    ds = yt.load( os.path.join(data_path, 'plt%05d/' %iteration) )
    all_data_level_0 = ds.covering_grid(level=0, 
                        left_edge=ds.domain_left_edge, 
                        dims=ds.domain_dimensions)
    
    if plotter == 'yt':
        sl = yt.SlicePlot(ds, slicing_direction, field)
        sl.set_log( field, False)
        sl.annotate_grids()
        # Show the new plot
        clear_output()
        sl.show()

    elif plotter == 'matplotlib':

        left_edge = ds.domain_left_edge.convert_to_mks()*1.e6
        right_edge = ds.domain_right_edge.convert_to_mks()*1.e6
        
        if ds.dimensionality == 3:
            if slicing_direction == 'x':
                n = int( ds.domain_dimensions[0]//2 )
                data2d = all_data_level_0[field][n, :, :]
                extent = [ left_edge[2], right_edge[2], left_edge[1], right_edge[1] ]
            elif slicing_direction == 'y':
                n = int( ds.domain_dimensions[1]//2 )
                data2d = all_data_level_0[field][:, n, :]
                extent = [ left_edge[2], right_edge[2], left_edge[0], right_edge[0] ]
            elif slicing_direction == 'z':
                n = int( ds.domain_dimensions[2]//2 )
                data2d = all_data_level_0[field][:, :, n]
                extent = [ left_edge[1], right_edge[1], left_edge[0], right_edge[0] ]
        elif ds.dimensionality == 2:
            data2d = all_data_level_0[field][:,:,0].T
            extent = [ left_edge[1], right_edge[1], left_edge[0], right_edge[0] ]

        plt.clf()
        plt.imshow( data2d.to_ndarray(), interpolation='nearest', cmap='viridis',
                  origin='lower', extent=extent )
        plt.colorbar()

# Interactive viewer

In [6]:
# Find iterations
file_list = glob.glob(os.path.join( data_path, 'plt?????') )
iterations = [ int(file_name[-5:]) for file_name in file_list ]

interact(plot_field, 
         iteration = IntSlider(min=min(iterations), max=max(iterations), step=iterations[1]-iterations[0]),
         field = ToggleButtons( options=['jx', 'jy', 'jz', 'Ex', 'Ey', 'Ez', 'Bx', 'By', 'Bz'], value='jz'),
         slicing_direction = ToggleButtons( options=[ 'x', 'y', 'z'], value='y'),
         plotter = ToggleButtons( options=['matplotlib', 'yt'] ) )

<function __main__.plot_field>